TF

In [1]:
import seismic_ml_kernel

In [2]:
#train (train_data_dir, validation_data_dir, image_size)
#test (validation_data_dir + 'noise/', image_size)

In [3]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

#mn (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

#ci (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [1]:
def create_images (data, png_name = None):
    import seismic_handler
    from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    
    image = seismic_handler.convert_to_image (data.T)
    im_data = img_to_array (image)

    fk_data = seismic_handler.fk (data)
    fk_image = seismic_handler.convert_to_image (fk_data)
    im_fk_data = img_to_array (fk_image)
    
    spec_data = seismic_handler.spectrogram (data)
    spec_image = seismic_handler.convert_to_image (spec_data.T)
    im_spec_data = img_to_array (spec_image)
    
    if png_name != None:
        image.save (png_name + '_data.png')
        fk_image.save (png_name + '_fk.png')
        spec_image.save (png_name + '_spec.png')
    
    return [im_data, im_fk_data, im_spec_data]
                
def seis_dataset (file_name, sorting, window, noisers):
    import numpy as np
    import seismic_handler
        
    images_dir = 'images/'

    import os
    if os.path.exists(images_dir):
        import shutil
        shutil.rmtree(images_dir)

    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    train_data_dir = images_dir + 'train/'
    validation_data_dir = images_dir + 'validation/'

    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)

    if not os.path.exists(validation_data_dir):
        os.makedirs(validation_data_dir)
        
    for i in range(len(noisers)):
        a = train_data_dir + str(i)
        if not os.path.exists(a):
            os.makedirs(a)
    
        a = validation_data_dir + str(i)
        if not os.path.exists(a):
            os.makedirs(a)
    
    handlers = [seismic_handler.SeismicPrestack(file_name, noiser) for noiser in noisers]
    gather_keys = [handler.getHeaderVals (sorting) for handler in handlers]
        
    X_train = [[],[],[]]
    y_train = []
    X_test = [[],[],[]] 
    y_test = []
      
    
    counter = 0
    nhandlers = len (handlers)
           
    for key in gather_keys[0]:
        partss = [handler.readGatherParts (key, window[0], window[1]) for handler in handlers]       
        sh = partss[0][0].shape
        
        for category in range(nhandlers):
            nparts = len (partss[category])
            for i in range(nparts):
                
                # just sparser of data
                if category != i % nhandlers:
                    continue

                data = partss[category][i]
                if data.shape != sh:
                    throw ('wrong shapes')
                
                train = False
                if i < nparts*0.8:
                    train = True
                    
                if train:
                    png_name = train_data_dir
                else:
                    png_name = validation_data_dir
                    
                png_name += str (category) + '/'    
                png_name += str(counter)
                png_name = None
                    
                data_repr = create_images (data, png_name)               

                if train:
                    for i in range (len(data_repr)):
                        X_train[i].append(data_repr[i])
                    y_train.append (category)
                else:
                    for i in range (len(data_repr)):
                        X_test[i].append(data_repr[i])
                    y_test.append(category)   
                
                counter += 1
    
    y_test = np.array(y_test)
    y_train = np.array(y_train)
    for i in range (len(X_train)):
        X_train[i] = np.array(X_train[i])
        X_test[i] = np.array(X_test[i])
        if len(X_train[i]) != len(X_train[0]):
            print ('len(X_train[i]) != len(X_train[0])')
            return
        if len(X_test[i]) != len(X_test[0]):
            print ('len(X_test[i]) != len(X_test[0])')
            return
    
    if len(X_train[0]) != len(y_train):
        print ('len(X_train) != len(y_train)')
        return
    if len(X_test[0]) != len(y_test):
        print ('len(X_test) != len(y_test)')
        return

    return (X_train, y_train), (X_test, y_test)

In [ ]:
import seismic_handler
    
#noisers = [None, seismic_handler.FreqNoiser()]
noisers = [None, 
           seismic_handler.BP(0.1, 0.15, 0.2, 0.25, 1), 
           seismic_handler.BP(0.2, 0.25, 0.3, 0.35, 1),
           seismic_handler.BP(0.3, 0.35, 0.4, 0.45, 1),
           seismic_handler.BP(0.4, 0.45, 0.5, 0.55, 1),
          ]
#noisers = [None, seismic_handler.BP(0.1, 0.15, 0.2, 0.25, 1)]

(X_train, y_train), (X_test, y_test) = seis_dataset ('data/prestack.sgy', 'original_field_record_number', [50, 0.5], noisers)

import seismic_ml_kernel
#seismic_ml_kernel.mn (X_train[0], y_train, X_test[0], y_test)
seismic_ml_kernel.ci_multi (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


num branches 3
image has 4600 pixels


C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:123: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:124: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:128: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
  conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:129

image has 2300 pixels
image has 2350 pixels
Vision models are ready
Classification model is ready
Classification model compiled


C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:191: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation


Train on 12226 samples, validate on 1359 samples
Epoch 1/200
 4544/12226 [==========>...................] - ETA: 630s - loss: 1.6150 - acc: 0.2194